In [2]:
import tensorflow as tf
from tensorflow import keras
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import keras as kr

import plotly.express as px
from plotly.subplots import make_subplots
import plotly.graph_objects as go


from sklearn.metrics import euclidean_distances

2022-12-02 10:38:30.417915: I tensorflow/core/platform/cpu_feature_guard.cc:193] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-critical operations:  AVX2 AVX512F AVX512_VNNI FMA
To enable them in other operations, rebuild TensorFlow with the appropriate compiler flags.
2022-12-02 10:38:30.555706: I tensorflow/core/util/util.cc:169] oneDNN custom operations are on. You may see slightly different numerical results due to floating-point round-off errors from different computation orders. To turn them off, set the environment variable `TF_ENABLE_ONEDNN_OPTS=0`.
2022-12-02 10:38:30.558814: W tensorflow/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'libcudart.so.11.0'; dlerror: libcudart.so.11.0: cannot open shared object file: No such file or directory
2022-12-02 10:38:30.558822: I tensorflow/stream_executor/cuda/cudart_stub.cc:29] Ignore above cudart dlerror if yo

In [3]:

df = pd.read_csv("/home/vmh/vmhdocs/Research/Inria/Anl/EnvDataAutoencoder/Data/Imputed_Metadata_metaGmetaT_by_OceanLayer.csv",sep=",")

selectcols = ["Latitude","Longitude","Depth.nominal","Temperature","Oxygen","ChlorophyllA","Carbon.total","Salinity","Gradient.Surface.temp","Fluorescence","CO3","HCO3","Density","PO4","NO3","Si","Alkalinity.total","Ammonium.5m","Lyapunov","NO2","Depth.Min.O2","NO2NO3","Nitracline","Brunt.Vaisala","Iron.5m","Depth.Max.O2","Okubo.Weiss"]
df_env = df[selectcols]

train_data = np.array(df_env)
print(train_data.shape)

inputlen = len(df_env.columns)
print(inputlen)

df_env.head(5)

(228, 27)
27


,Latitude,Longitude,Depth.nominal,Temperature,Oxygen,ChlorophyllA,Carbon.total,Salinity,Gradient.Surface.temp,Fluorescence,...,Ammonium.5m,Lyapunov,NO2,Depth.Min.O2,NO2NO3,Nitracline,Brunt.Vaisala,Iron.5m,Depth.Max.O2,Okubo.Weiss
0,35.7590,14.2574,5,21.486650,208.245000,0.013344,0.01030,37.895750,1.05130,1.091875,...,0.009120,0.066158,0.018330,221,0.118000,193,61,0.001388,60,-0.124391
1,33.9179,32.8980,5,20.442500,207.613550,0.067706,0.00500,39.423110,0.24369,1.176500,...,0.003270,0.097233,0.001250,151,0.045500,193,59,0.001388,55,-0.309549
2,37.0510,1.9378,9,23.814100,217.809833,0.189348,0.00695,37.522000,1.03750,1.183333,...,0.015245,0.000000,0.044500,167,0.143250,193,45,0.001388,37,-1.381300
3,42.2038,17.7150,5,17.211742,221.782083,0.103935,0.00500,38.228050,1.61570,1.465000,...,0.001431,0.078584,0.009125,115,0.040875,193,3,0.001388,37,-0.354776
4,39.3888,19.3905,5,18.319192,217.809833,0.104797,0.00500,38.185333,2.00910,1.465000,...,0.002327,0.000000,0.004250,174,0.035250,193,41,0.001388,37,0.369749


In [4]:
autoencoder = kr.models.load_model('/home/vmh/vmhdocs/Research/Inria/Anl/EnvDataAutoencoder/Code/autoenc001/autoencoder_envdata.h5')
autoencoder.summary()

Model: "model_1"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 input_2 (InputLayer)        [(None, 27)]              0         
                                                                 
 dense_6 (Dense)             (None, 514)               14392     
                                                                 
 batch_normalization_5 (Batc  (None, 514)              2056      
 hNormalization)                                                 
                                                                 
 dense_7 (Dense)             (None, 257)               132355    
                                                                 
 batch_normalization_6 (Batc  (None, 257)              1028      
 hNormalization)                                                 
                                                                 
 dense_8 (Dense)             (None, 3)                 774 

2022-12-02 10:38:38.185974: W tensorflow/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'libcuda.so.1'; dlerror: libcuda.so.1: cannot open shared object file: No such file or directory
2022-12-02 10:38:38.185991: W tensorflow/stream_executor/cuda/cuda_driver.cc:263] failed call to cuInit: UNKNOWN ERROR (303)
2022-12-02 10:38:38.186004: I tensorflow/stream_executor/cuda/cuda_diagnostics.cc:156] kernel driver does not appear to be running on this host (vmhpc): /proc/driver/nvidia/version does not exist
2022-12-02 10:38:38.186148: I tensorflow/core/platform/cpu_feature_guard.cc:193] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-critical operations:  AVX2 AVX512F AVX512_VNNI FMA
To enable them in other operations, rebuild TensorFlow with the appropriate compiler flags.


In [6]:
# Fit with the entire dataset.
checkpoint_filepath = './modelcheckpoint'
chkpt = tf.keras.callbacks.ModelCheckpoint(
    filepath=checkpoint_filepath,
    save_best_only=True,
    monitor='loss',
    verbose=1)

tb = kr.callbacks.TensorBoard(log_dir="./model_tb",update_freq="epoch")
earlystop = tf.keras.callbacks.EarlyStopping(monitor='loss', patience=500)

autoencoder.fit(train_data,train_data,shuffle=True, epochs=50000, 
                            batch_size=50, verbose=0,callbacks=[chkpt,tb,earlystop])




Epoch 1: loss improved from inf to 509.06201, saving model to ./modelcheckpoint
INFO:tensorflow:Assets written to: ./modelcheckpoint/assets

Epoch 2: loss did not improve from 509.06201

Epoch 3: loss improved from 509.06201 to 431.97302, saving model to ./modelcheckpoint
INFO:tensorflow:Assets written to: ./modelcheckpoint/assets

Epoch 4: loss improved from 431.97302 to 386.45740, saving model to ./modelcheckpoint
INFO:tensorflow:Assets written to: ./modelcheckpoint/assets

Epoch 5: loss did not improve from 386.45740

Epoch 6: loss improved from 386.45740 to 376.57416, saving model to ./modelcheckpoint
INFO:tensorflow:Assets written to: ./modelcheckpoint/assets

Epoch 7: loss did not improve from 376.57416

Epoch 8: loss did not improve from 376.57416

Epoch 9: loss did not improve from 376.57416

Epoch 10: loss did not improve from 376.57416

Epoch 11: loss did not improve from 376.57416

Epoch 12: loss improved from 376.57416 to 360.58920, saving model to ./modelcheckpoint
INFO:t